# Alanine Scanning

## Without *mcdrd files 
Prior to deploying the pipeline, as done below, the user should make use of the `run_simulation_gpu.sh` script. The user is free to customize their input files (*.in) as presented in the AMBER manual. 

Note: The provided `tmp_input_file.txt` assumes all `*.mcdrd` files are located in the same folder as the `htms_pipeline.py` call.

## With *mcdrd files

To run the Alanine Scanning pipeline edit the tmp_input_file.txt according your desired specifications. In this example we will be looking at 3 mutations: T:470:A E:471:A I:472:A

For the purposes of demonstrating the workflow we are calling htms_pipeline.py from the example direcotry. Moreover, we are making use of the --just_build option to not actully run our mmpbsa.py calls. This is done to showcase the standard output format. 

`HTMS_Amber\examples$ python ..\HTMS_Amber\htms_pipeline.py --input_file .\tmp_input_file.txt --just_build`

Simply call the script as below to delpoy the pipeline properly:

`HTMS_Amber\examples$ python ..\HTMS_Amber\htms_pipeline.py --input_file .\tmp_input_file.txt`

We note `mmpbsa.in` in placed within the directory the pipeline is deployed from and shared amoung all runs.  


# Non-Alanine Mutations

In order to handle non-Alanine mutations we opt to incorporate a call to MODELLER for mutated pdb file generation. 

In [1]:
import subprocess 
import pandas as pd
import numpy as np
import os
import shutil
import glob

In [2]:
amino_acids = {
    'A': 'ALA',  # Alanine
    'R': 'ARG',  # Arginine
    'N': 'ASN',  # Asparagine
    'D': 'ASP',  # Aspartic Acid
    'C': 'CYS',  # Cysteine
    'E': 'GLU',  # Glutamic Acid
    'Q': 'GLN',  # Glutamine
    'G': 'GLY',  # Glycine
    'H': 'HIS',  # Histidine
    'I': 'ILE',  # Isoleucine
    'L': 'LEU',  # Leucine
    'K': 'LYS',  # Lysine
    'M': 'MET',  # Methionine
    'F': 'PHE',  # Phenylalanine
    'P': 'PRO',  # Proline
    'S': 'SER',  # Serine
    'T': 'THR',  # Threonine
    'W': 'TRP',  # Tryptophan
    'Y': 'TYR',  # Tyrosine
    'V': 'VAL',  # Valine
}


In [3]:
E417N = [("417", amino_acids["N"])]
# E417T = [("417", amino_acids["T"])]
# S477N = [("477", amino_acids["N"])]
# E484K = [("484", amino_acids["K"])]
# N501Y = [("501", amino_acids["Y"])]
# K417N_E484K = [("484", amino_acids["K"]), ("417", amino_acids["N"])]
# K417T_E484K = [("484", amino_acids["K"]), ("417", amino_acids["T"])]
E484K_N501Y = [("484", amino_acids["K"]), ("501", amino_acids["Y"])]

mutations_dict ={"E417N": E417N, 
                #  "E417T": E417T
                #  "S477N": S477N, 
                #  "E484K": E484K, 
                #  "N501Y": N501Y,
                #  "K417N_E484K": K417N_E484K,
                #  "K417T_E484K": K417T_E484K,
                 "E484K_N501Y": E484K_N501Y}

In [4]:
import sys
from  pathlib import Path
import os 
sys.path.append((Path(os.getcwd()).parent).as_posix()) 
from HTMS_Amber import _non_ala_mut

In [6]:
for mutation , mut_input_list in mutations_dict.items() :
	#turn our arr of tuples into a input for sys.argv (kinda dumb, but will work)
	flattened_list = [str(item) for tup in mut_input_list for item in tup] 

	formatted_string_mut_in = ' '.join(flattened_list)

	# modeller_str = make_model_str + " " + mut_input_list + "asdasd" + " A"
	dir_str = "_" + mutation

	if not os.path.exists(dir_str):
		os.makedirs(dir_str)

	#subprocess.run(f"{make_model_str} {mutation} A {formatted_string_mut_in}")
	_non_ala_mut.general_mutate(modelname="6m0j_noHet", mutation=mutation)

	#move the file into the dir. 
	pdb_file = f"6m0j_noHet{mutation}.pdb"
	subprocess.run(["move", pdb_file, dir_str], shell=True)

	os.chdir(dir_str) #step into dir, makes passing outputs easier 
	print(os.getcwd())
	sim_set_up_str = ["python", "../sim_set_up.py",  pdb_file]
	subprocess.run(sim_set_up_str)

	#subprocess.run(['sed', '-i', 's/\r$//', 'all_process.sh'])
	subprocess.run(["C:/Program Files/Git/usr/bin/sed", "-i", "s/\r$//", "all_process.sh"])
	#subprocess.run(['dos2unix', 'all_process.sh'])
	subprocess.run(["C:/Program Files/Git/usr/bin/sed", "-i", "s/\r$//", "'run_MMPBSA.sh'"])
	#subprocess.run(['dos2unix', 'run_MMPBSA.sh'])
	os.chdir("..")
	print(os.getcwd())
	#place in files into dir 
	for in_file in glob.glob("gen_in_files\*.in"):
		shutil.copy(in_file, dir_str)

read_to_681_> topology.submodel read from topology file:        3
readlinef__W> File: 6m0j_noHet.pdb, Line: 6960
              Modeller will only read the first 80 characters of this line.

readlinef__W> File: 6m0j_noHet.pdb, Line: 6960
              Modeller will only read the first 80 characters of this line.

  #      ALGNMT CODE

  1  6m0j_noHet
  2  6m0j_noHet
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      791
Number of all, selected real atoms                :     6408       8
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :    31677      52
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number